In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

def classification_training(data):
    df = data.copy()
    # Трансформируем целевую переменную в бинарную
    # значения < 15 -> 0, >= 15 -> 1
    df['mental_wellness_binary'] = np.where(df['mental_wellness_index_0_100'] < 15, 0, 1)
    
    # Подготавливаем признаки и целевую переменную
    exclude_cols = ['user_id', 'gender', 'occupation', 'work_mode', 
                   'mental_wellness_index_0_100', 'mental_wellness_binary']
    
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    X = df[feature_cols]
    y = df['mental_wellness_binary']
    
    # Разделяем данные на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Создаем экземпляр класса случайного леса
    rf_classifier = RandomForestClassifier(random_state=42)
    
    # Создаем словарь-сетку для поиска гиперпараметров
    param_grid = {
        'n_estimators': [50, 100, 200],           # Количество деревьев
        'max_depth': [None, 10, 20, 30],          # Максимальная глубина деревьев
        'min_samples_split': [2, 5, 10],          # Минимальное количество samples для разделения
        'min_samples_leaf': [1, 2, 4],            # Минимальное количество samples в листе
        'max_features': ['sqrt', 'log2']          # Количество признаков для разделения
    }
    
    # Выполняем поиск оптимальных гиперпараметров
    grid_search = GridSearchCV(
        estimator=rf_classifier,
        param_grid=param_grid,
        cv=5,                    # 5-кратная кросс-валидация
        scoring='f1',           # Оптимизируем по F1-score
        n_jobs=-1,              # Используем все доступные ядра
        verbose=1               # Вывод прогресса
    )
    
    grid_search.fit(X_train, y_train)
   
    # Выводим оптимальный набор гиперпараметров
    best_params = grid_search.best_params_
    print("Best parameters found:")
    for param, value in best_params.items():
        print(f"  {param}: {value}")
    
    # Обучаем финальную модель с оптимальными параметрами
    final_model = grid_search.best_estimator_
    final_model.fit(X_train, y_train)  # Переобучаем на всех тренировочных данных
    
    # Предсказания и оценка модели
    y_pred = final_model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"RF: {accuracy:.4f}; {f1:.4f}")

#data = pd.read_csv("cleaned_DB_3.csv")
#classification_training(data)